<a href="https://colab.research.google.com/github/PhongCT1105/AI_Tutor_Transformer_Model/blob/main/AI_Tutor_Transfomer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

#AI Tutor for Deep Learning Class built from scratch

#I. Building model and Training

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader



#1. Import dataset used for training

In [4]:
from datasets import load_dataset
original_dataset = load_dataset("allenai/sciq")

print(original_dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.99M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/339k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})


In [5]:
# Feature Engineering:
def combined_features(dataset):
  return {
      "question": dataset["question"],
      "answer": "The answer is " + dataset["correct_answer"] + ". Explanation: " + dataset["support"]
  }

dataset = original_dataset.map(combined_features)
dataset = dataset.remove_columns(["distractor3", "distractor1", "distractor2", "correct_answer", "support"])

# Print feature combined:
print(dataset)
print(dataset["train"][0])

Map:   0%|          | 0/11679 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1000
    })
})
{'question': 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?', 'answer': 'The answer is mesophilic organisms. Explanation: Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'}


#2. Tokenizer


###2.1 Vocabulary creation

In [46]:
from collections import Counter

def vocab_mapping(dataset, freq=1, part = "train"):

  # Counting word frequency
  counter = Counter()
  for sample in dataset[part]:
    for word in sample["question"].split():
      counter[word] += 1
    for word in sample["answer"].split():
      counter[word] += 1

  # Creating word collection for encode
  vocab = {
      "<PAD>": 0, # Paddinng token
      "<UNK>": 1, # Unknown token
      "<BOS>": 2, # Beginning of sequence
      "<SEP>": 3, # Seperate token between question and answer
      "<EOS>": 4, # Ending of sequence
  }

  # Unique ID for each word
  for word, count in counter.items():
    if count >= freq and word not in vocab:
      vocab[word] = len(vocab)


  return vocab

vocab = vocab_mapping(dataset, 1, "train")
print(vocab)

{'<PAD>': 0, '<UNK>': 1, '<BOS>': 2, '<SEP>': 3, '<EOS>': 4, 'What': 5, 'type': 6, 'of': 7, 'organism': 8, 'is': 9, 'commonly': 10, 'used': 11, 'in': 12, 'preparation': 13, 'foods': 14, 'such': 15, 'as': 16, 'cheese': 17, 'and': 18, 'yogurt?': 19, 'The': 20, 'answer': 21, 'mesophilic': 22, 'organisms.': 23, 'Explanation:': 24, 'Mesophiles': 25, 'grow': 26, 'best': 27, 'moderate': 28, 'temperature,': 29, 'typically': 30, 'between': 31, '25°C': 32, '40°C': 33, '(77°F': 34, '104°F).': 35, 'are': 36, 'often': 37, 'found': 38, 'living': 39, 'or': 40, 'on': 41, 'the': 42, 'bodies': 43, 'humans': 44, 'other': 45, 'animals.': 46, 'optimal': 47, 'growth': 48, 'temperature': 49, 'many': 50, 'pathogenic': 51, 'mesophiles': 52, '37°C': 53, '(98°F),': 54, 'normal': 55, 'human': 56, 'body': 57, 'temperature.': 58, 'Mesophilic': 59, 'organisms': 60, 'have': 61, 'important': 62, 'uses': 63, 'food': 64, 'preparation,': 65, 'including': 66, 'cheese,': 67, 'yogurt,': 68, 'beer': 69, 'wine.': 70, 'phenome

###2.2 Tokenize

In [47]:
MAX_LENGTH = 128

def tokenize_map(row):
    # 1) Tokenize question and answer
    q_ids = []
    for word in row["question"].split():
      q_ids.append(vocab.get(word, vocab["<UNK>"]))
    a_ids = []
    for word in row["answer"].split():
      a_ids.append(vocab.get(word, vocab["<UNK>"]))

    # 2) Add special token
    seq = [vocab["<BOS>"]] + q_ids + [vocab["<SEP>"]] + a_ids + [vocab["<EOS>"]]
    input = seq[:-1]
    label = seq[1:]

    # 3) Add padding

    # i) PAD = 0 normally for input
    num_pad_i = MAX_LENGTH - len(input)
    if num_pad_i > 0:
      input += num_pad_i * [vocab["<PAD>"]]
    elif num_pad_i < 0:
      input = input[:MAX_LENGTH]

    # ii) PAD = -100 for output because we don't want to penalty model for predicting PAD
    num_pad_o = MAX_LENGTH - len(label)
    if num_pad_o > 0:
      label += num_pad_o * [-100]
    elif num_pad_o < 0:
      label = label[:MAX_LENGTH]

    # 4) Add attention mask to recognize real and PAD token
    attention_mask = []
    for word in input:
      if word == vocab["<PAD>"]:
        attention_mask.append(0)
      else:
        attention_mask.append(1)

    return {
        "input_ids": input,
        "attention_mask": attention_mask,
        "labels": label,
    }

data_train = dataset["train"].map(tokenize_map, remove_columns=["question","answer"])
print(data_train)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 11679
})


In [48]:
# Visualize one sample

print(data_train["input_ids"][0])
print(data_train["attention_mask"][0])
print(data_train["labels"][0])

[2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 7, 14, 15, 16, 17, 18, 19, 3, 20, 21, 9, 22, 23, 24, 25, 26, 27, 12, 28, 29, 30, 31, 32, 18, 33, 34, 18, 35, 25, 36, 37, 38, 39, 12, 40, 41, 42, 43, 7, 44, 40, 45, 46, 20, 47, 48, 49, 7, 50, 51, 52, 9, 53, 54, 42, 55, 56, 57, 58, 59, 60, 61, 62, 63, 12, 64, 65, 66, 67, 68, 69, 18, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 7, 14, 15, 16, 17, 18, 19, 3, 20, 21, 9, 22, 23, 24, 25, 26, 27, 12, 28, 29, 30, 31, 32, 18, 33, 34, 18, 35, 25, 36, 37, 38, 39

In [52]:
# Convert into dataset

# Convert into tensor
data_train.set_format(
    type = "torch",
    columns = data_train.column_names
)

train_loader = DataLoader(
    data_train,
    batch_size = 32,
    shuffle = True,
)

print(train_loader)

###3. Word Embedding & Positional Encoding